# Chargement des données en mémoire

## Méthode 1 
écrire une fonction en python natif pour charger le fichier *covid_data.csv* dans un tableau de tableaux pour chaque entrée du ficher  

In [32]:
import csv

def charger_fichier_csv(nom_fichier):
    tableau = []
    with open(nom_fichier, newline='') as f:
        lecteur = csv.reader(f, delimiter=',', quotechar='"')
        for ligne in lecteur:
            tableau.append(ligne)
    return tableau

tableau = charger_fichier_csv("covid_data.csv")

## Méthode 2 
charger les données en utilisant numpy ou pandas

In [34]:
# numpy/pandas data loader
import numpy as np
import pandas as pd

df = pd.read_csv("covid_data.csv")
df.head()



,,,continent;country;date;total_cases;total_deaths;total_icu_patients;total_hosp_patients;total_tests;total_vaccinations;people_vaccinated;people_fully_vaccinated;median_age;aged_65_older;aged_70_older;population
Asia;Afghanistan;2020-01-03;;;;;;;;;18,6;2,581;1,337;41128772
Asia;Afghanistan;2020-01-04;;;;;;;;;18,6;2,581;1,337;41128772
Asia;Afghanistan;2020-01-05;;;;;;;;;18,6;2,581;1,337;41128772
Asia;Afghanistan;2020-01-06;;;;;;;;;18,6;2,581;1,337;41128772
Asia;Afghanistan;2020-01-07;;;;;;;;;18,6;2,581;1,337;41128772


comparer la performance des deux méthodes (temps d'éxecution du chargement).

# Console de statistiques

Créer une classe `CmdStat` responsable de répondre, avec des valeurs ou des plots, aux intérogations de l'utilisateur en ligne de commande. 
- La base de données de la classe `CmdStat` est la structure de données (de votre choix) contenant les informations du ficher *covid_data.csv*. Cette structure de données est un attribut de la classe `CmdStat`.
- La classe `CmdStat` contient plusieurs méthodes :
    - Une méthode `terminal` pour gérer les entrées (commandes) et les sorties (affichage).
    - Une méthode `parser` pour analyser la commande de l'utilisateur. Cette méthode doit vérifier la syntaxe et extraire les paramètres à passer à/aux méthode(s) responsable(s) de générer une réponse à afficher.
    - Une ou un ensesembe de méthodes pour accéder à la structure de données et récupérer l'information demandée.
- Les commandes sont sous la forme suivante : `covidstat [--info, --hist, --plt] <information_1, ..., information_n> [--aggregation (optional) <max, min, avg>] [--location (optional)] <location_1, ..., location_m> [--time (optional) <dates, days, months, years>] <value>`
- Les informations accessibles sont :
    + total_cases
    + total_deaths
    + total_icu_patients
    + total_hosp_patients
    + total_tests
    + total_positives
    + new_cases
    + new_deaths
    + new_icu_patients
    + new_hosp_patients
    + new_tests
    + new_positives
- Les localisations sont celles présentes dans les données.
- L'option temps est pour choisir la periode par inetrvalle de dates ou par nombre de derniers jours, mois, ou années. 
- Voici les tois familles de commandes possibles :
    1. **info** : Pour cette famille, l'aggrégation et le temps sont obligatoires. exemples :
        + `covidstat --info total_cases --aggregation max --time days 7` output : le maximum des cas totales dans les 7 derniers jours est xx à xx.
        + `covidstat --info total_tests --aggregation avg --time months 2 --location Italia` output : la moyenne des tests effectués en Italia les 2 derniers mois est xx. 
    2. **hist** : L'affichage est un ou plusieurs histogrammes. L'option aggregtion n'est pas acceptée. La seule option temps acceptée est en spécifiant un intervalle de dates. exemple :
        + `covidstat --hist new_deaths --time 10-10-2021 10-12-2021 --location France Belgium China India`
    3. **plot** : L'affichage est une ou plusieurs courbes. L'option aggregtion n'est pas acceptée. L'absence de l'option localisation est équivalente à lister l'ensemble des pays. L'absence de l'option temps est équivalente à l'intervalle globale.

Ne pas oublier de mettre en place une gestion des exceptions et d'écrire un code conforme aux bonnes pratiques mentionnées durant ce module.


In [35]:
import csv
import argparse
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

class CmdStat():
    def __init__(self, fichier):
        self.donnees = self.charger_fichier_csv(fichier)

    def terminal(self):
        while True:
            commande = input("covidstat> ")
            if commande == "exit":
                break
            try:
                resultat = self.parser(commande)
                print(resultat)
            except Exception as e:
                print("Erreur : ", e)

    def parser(self, commande):
        parser = argparse.ArgumentParser(prog="covidstat", description="Afficher des statistiques COVID-19.")
        parser.add_argument("--info", choices=["total_cases", "total_deaths", "total_icu_patients", "total_hosp_patients", "total_tests", "total_positives", "new_cases", "new_deaths", "new_icu_patients", "new_hosp_patients", "new_tests", "new_positives"], help="Information à afficher.")
        parser.add_argument("--hist", choices=["new_cases", "new_deaths", "new_icu_patients", "new_hosp_patients", "new_tests", "new_positives"], help="Afficher un histogramme.")
        parser.add_argument("--plt", choices=["total_cases", "total_deaths", "total_icu_patients", "total_hosp_patients", "total_tests", "total_positives", "new_cases", "new_deaths", "new_icu_patients", "new_hosp_patients", "new_tests", "new_positives"], help="Afficher un graphique.")
        parser.add_argument("--aggregation", choices=["max", "min", "avg"], help="Type d'agrégation.")
        parser.add_argument("--location", nargs="+", help="Localisation.")
        parser.add_argument("--time", nargs=2, help="Période de temps.")
        parser.add_argument("value", type=int, help="Valeur.")
        args = parser.parse_args(commande.split())

        if args.info:
            if not args.aggregation or not args.time or not args.location:
                raise ValueError("Les options --aggregation, --time et --location sont obligatoires pour l'option --info.")
            resultat = self.info(args.info, args.aggregation, args.time, args.location, args.value)
        elif args.hist:
            if not args.time:
                raise ValueError("L'option --time est obligatoire pour l'option --hist.")
            resultat = self.hist(args.hist, args.time, args.location)
        elif args.plt:
            resultat = self.plt(args.plt, args.time, args.location)
        else:
            raise ValueError("L'option --info, --hist ou --plt est obligatoire.")

        return resultat

    def charger_fichier_csv(self, nom_fichier):
        donnees = []
        with open(nom_fichier, newline='') as f:
            lecteur = csv.reader(f, delimiter=',', quotechar='"')
            for ligne in lecteur:
                donnees.append(ligne)
        return donnees

    def extraire_donnees(self, info, location, debut, fin):
        donnees_filtrees = []
        for ligne in self.donnees[1:]:
            if ligne[1] == info and ligne[0] in location:
                date = datetime.strptime(ligne[2], "%Y-%m-%d")
                if debut <= date <= fin:
                    donnees_filtrees.append((ligne[0], date, int(ligne[3])))
        return donnees_filtrees

    def info(self, info, aggregation, time, location, value):
        debut, fin = self.calculer_dates(time)
        donnees_filtrees = self.extraire_donnees(info, location, debut, fin)
        if not donnees_filtrees:
            raise ValueError("Aucune donnée trouvée pour les paramètres spécifiés.")
        if aggregation == "max":
            resultat = max(donnees_filtrees, key=lambda x: x[2])
        elif aggregation == "min":
            resultat = min(donnees_filtrees, key=lambda x: x[2])
        elif aggregation == "avg":
            somme = 0
            for donnee in donnees_filtrees:
                somme += donnee[2]
            resultat = somme / len(donnees_filtrees)
        return f"{aggregation.capitalize()} des {info.replace('_', ' ')} entre {debut.date()} et {fin.date()} pour {', '.join(location)} est {resultat}."

    def hist(self, info, time, location):
        debut, fin = self.calculer_dates(time)
        donnees_filtrees = self.extraire_donnees(info, location, debut, fin)
        if not donnees_filtrees:
            raise ValueError("Aucune donnée trouvée pour les paramètres spécifiés.")
        donnees_par_location = {}
        for donnee in donnees_filtrees:
            if donnee[0] not in donnees_par_location:
                donnees_par_location[donnee[0]] = []
            donnees_par_location[donnee[0]].append(donnee[2])
        for location, donnees in donnees_par_location.items():
            plt.hist(donnees, bins=10, alpha=0.5, label=location)
        plt.legend(loc='upper right')
        plt.title(f"Histogramme des {info.replace('_', ' ')} entre {debut.date()} et {fin.date()}")
        plt.show()

    def plt(self, info, time, location):
        debut, fin = self.calculer_dates(time)
        if not location:
            location = list(set([ligne[0] for ligne in self.donnees[1:]]))
        donnees_par_location = {}
        for loc in location:
            donnees_filtrees = self.extraire_donnees(info, [loc], debut, fin)
            if donnees_filtrees:
                donnees_par_location[loc] = donnees_filtrees
        if not donnees_par_location:
            raise ValueError("Aucune donnée trouvée pour les paramètres spécifiés.")
        for location, donnees in donnees_par_location.items():
            x = [donnee[1] for donnee in donnees]
            y = [donnee[2] for donnee in donnees]
            plt.plot(x, y, label=location)
        plt.legend(loc='upper left')
        plt.title(f"{info.replace('_', ' ')} entre {debut.date()} et {fin.date()}")
        plt.show()

    def calculer_dates(self, time):
        if time[0].isdigit():
            debut = datetime.now() - timedelta(days=int(time[0]))
        else:
            debut = datetime.strptime(time[0], "%d-%m-%Y")
        if time[1].isdigit():
            fin = datetime.now() - timedelta(days=int(time[1]))
        else:
            fin = datetime.strptime(time[1], "%d-%m-%Y")
        return debut, fin

## Test de la class `CmdStat`

In [37]:
covid_stat = CmdStat("covid_data.csv")
covid_stat.terminal()


usage: covidstat [-h]
                 [--info {total_cases,total_deaths,total_icu_patients,total_hosp_patients,total_tests,total_positives,new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                 [--hist {new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                 [--plt {total_cases,total_deaths,total_icu_patients,total_hosp_patients,total_tests,total_positives,new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                 [--aggregation {max,min,avg}]
                 [--location LOCATION [LOCATION ...]] [--time TIME TIME]
                 value
covidstat: error: the following arguments are required: value


SystemExit: 2

Mettre la classe `Cmd Stat` ainsi que les potentielles méthodes utilitaires dans un module *covidstat.py* et l'invoquer dans le notebook.

In [41]:
from covidstat import CmdStat

covid_stat = CmdStat("covid_data.csv")
covid_stat.terminal()

usage: ipykernel_launcher.py [-h]
                             [--info {total_cases,total_deaths,total_icu_patients,total_hosp_patients,total_tests,total_positives,new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                             [--hist {new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                             [--plt {total_cases,total_deaths,total_icu_patients,total_hosp_patients,total_tests,total_positives,new_cases,new_deaths,new_icu_patients,new_hosp_patients,new_tests,new_positives}]
                             [--aggregation {max,min,avg}]
                             [--location LOCATION [LOCATION ...]]
                             [--time TIME TIME]
                             value
ipykernel_launcher.py: error: the following arguments are required: value


SystemExit: 2